Goal: pull out 1 buoy record that has ssc&chla and search for matching icesat2 granules

To do:
- Load info for 1 relevant station
- Find all matching atl granules and save times from file names
- Compare times to see if any matching buoy times

In [3]:
%pip install --quiet erddapy

Note: you may need to restart the kernel to use updated packages.


In [1]:
# from erddapy import ERDDAP
import pandas as pd
import numpy as np
from sliderule import sliderule, icesat2, earthdata
from datetime import datetime, timezone, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sliderule.init()

Client (version (4, 5, 3)) is out of date with the server (version (4, 6, 2))


False

In [2]:
def fname2datetime(fname):
    y = int(fname[6:10])
    m = int(fname[10:12])
    d = int(fname[12:14])
    H = int(fname[14:16])
    M = int(fname[16:18])
    S = int(fname[18:20])

    t = datetime(y,m,d,H,M,S, tzinfo=timezone.utc)
    return t

def buoy_bound_box(lat,lon,buffer_km):
    # define a buffer distance around the buoy to search for icesat-2 data
    lat_buff = buffer_km/111 # convert buffer distance to frac of 1 deg lat
    lon_buff = buffer_km/(111*np.cos(lat*np.pi/180)) # convert buffer distance to frac of 1 deg lon
    # define bounding box around the buoy (WSEN)
    # example: bbox = [-108.3, 39.2, -107.8, 38.8]
    # bbox = [lon-lon_buff,lat+lat_buff,lon+lon_buff,lat-lat_buff]
    # region = sliderule.toregion(bbox)
    minx = lon - lon_buff
    miny = lat - lat_buff
    maxx = lon + lon_buff
    maxy = lat + lat_buff

    poly = [{'lon': minx, 'lat': miny},
            {'lon': maxx, 'lat': miny},
            {'lon': maxx, 'lat': maxy},
            {'lon': minx, 'lat': maxy},
            {'lon': minx, 'lat': miny}] # Closing the loop by repeating the first point
    return poly


In [3]:
# load a list of all the relevant ERDDAPs and their urls
FF = pd.read_pickle("labeled_relevant_stations.pkl")
FF = FF[(FF.buoy == True) & (FF.radiation==True)]
FF.reset_index(drop=True, inplace=True)

FF["geospatial_lat_min"] = pd.to_numeric(FF["geospatial_lat_min"])
FF["geospatial_lon_min"] = pd.to_numeric(FF["geospatial_lon_min"])
FF["geospatial_lat_max"] = pd.to_numeric(FF["geospatial_lat_max"])
FF["geospatial_lon_max"] = pd.to_numeric(FF["geospatial_lon_max"])
FF

,sites,url,time_coverage_start,time_coverage_end,geospatial_lat_min,geospatial_lat_max,geospatial_lon_min,geospatial_lon_max,geospatial_lat_units,geospatial_lon_units,check_sum,buoy,glider,ship,turb,ssc,phot,chl,turbid,photo,radiation
0,mpoPmzaVikingBuoysTimeseries,https://erddap.ogsl.ca/erddap/,2020-07-09T05:00:00Z,2024-07-01T23:00:00Z,44.349300,49.734700,-124.771500,-52.586200,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True
1,bcodmo_dataset_737163,https://erddap.bco-dmo.org/erddap/,1989-07-29T19:00:00Z,NaN,22.750000,22.750000,-158.000000,-158.000000,degrees_north,degrees_east,1.0,True,False,True,True,False,True,True,False,True,True
2,bcodmo_dataset_3093,https://erddap.bco-dmo.org/erddap/,1995-11-08T10:12:00Z,NaN,10.492000,10.683000,-64.735000,-64.367000,degrees_north,degrees_east,1.0,True,False,True,True,False,True,True,True,True,True
3,edu_humboldt_humboldt,https://erddap.sensors.ioos.us/erddap/,2013-02-14T17:15:00Z,2024-07-01T21:45:00Z,40.777500,40.777500,-124.196520,-124.196520,degrees_north,degrees_east,1.0,True,False,False,True,False,True,True,True,True,True
4,nerrs_kachomet,https://erddap.sensors.ioos.us/erddap/,2003-06-01T09:15:00Z,2024-05-28T21:00:00Z,59.600930,59.600930,-151.409500,-151.409500,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True
5,ism-glos-obs_46,https://erddap.sensors.ioos.us/erddap/,2023-05-05T18:15:00Z,2024-07-01T18:20:00Z,46.968730,46.968730,-88.397680,-88.397680,degrees_north,degrees_east,1.0,True,False,False,False,False,True,False,False,True,True
6,edu_humboldt_humboldt,http://erddap.cencoos.org/erddap/,2013-02-14T17:15:00Z,2024-07-01T23:45:00Z,40.777500,40.777500,-124.196520,-124.196520,degrees_north,degrees_east,1.0,True,False,False,True,False,True,True,True,True,True
7,bios_jack-20190725T1346-delayed,https://gliders.ioos.us/erddap/,2019-07-25T13:58:08Z,2019-08-17T16:15:26Z,31.615136,32.359756,-64.570206,-63.869049,degrees_north,degrees_east,1.0,True,True,False,False,False,True,True,False,True,True
8,bios_jack-20190917T0241-delayed,https://gliders.ioos.us/erddap/,2019-09-17T02:09:41Z,2019-10-21T09:47:37Z,31.664385,31.862342,-64.167289,-63.887414,degrees_north,degrees_east,1.0,True,True,False,False,False,True,True,False,True,True
9,bios_jack-20220627T1351-delayed,https://gliders.ioos.us/erddap/,2022-06-27T13:27:55Z,2022-08-19T15:30:24Z,32.007419,32.362515,-64.597226,-63.838511,degrees_north,degrees_east,1.0,True,True,False,False,False,True,True,False,True,True


In [ ]:
# # example with no looping (next cell is loops)
# # make an ROI within a 0.1 km range of buoy
# jj = 2
# poly = buoy_bound_box(FF['geospatial_lat_min'][jj],
#                       FF['geospatial_lon_min'][jj],
#                       0.1)

# # search CMR for ATL03 granules in the bounding box
# grns = earthdata.cmr(short_name="ATL03",
#                      polygon=poly,
#                      version='006')

# # save the times for each granule as a datetime object
# icesat_times = [fname2datetime(fname) for fname in grns]
# print(len(icesat_times))

# # now check if buoy data exists for these granule times (loop)
# kk = 5
# t = icesat_times[kk]
# filename = grns[kk]
# print(filename)
# # make erddap request for all data within +/- 1h of the icesat-2 granule
# e = ERDDAP(server=FF['url'][jj],
#            protocol="tabledap",
#            response="csv")
# e.dataset_id = FF['sites'][jj]
# t_start = (t-timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S+00:00")
# t_end = (t+timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S+00:00")
# e.constraints = {"time>=": t_start,
#                  "time<=": t_end}

# # try to download the associated buoy data
# buoy = e.to_pandas(parse_dates=True)

# # try to download associated photons
# t_start = (t-timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
# t_end = (t+timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
# poly = buoy_bound_box(FF['geospatial_lat_min'][jj],
#                       FF['geospatial_lon_min'][jj],
#                       2)
# parms = {"poly": poly,
#          "t0": t_start,
#          "t1": t_end,
#          "srt": icesat2.SRT_OCEAN}

# atl_gdb = icesat2.atl03sp(parms)
# len(atl_gdb)

In [5]:
# loop through each buoy asset:
for jj in range(len(FF)):
    # define a search region around the buoy 
    lat = FF['geospatial_lat_min'][jj]
    lon = FF['geospatial_lon_min'][jj]
    poly = buoy_bound_box(lat,lon,0.5)

    # search CMR for ATL03 granules in the bounding box
    grns = earthdata.cmr(short_name="ATL03",
                         polygon=poly,
                         version='006')
    # save the times for each granule as a datetime object
    icesat_times = [fname2datetime(fname) for fname in grns]

    # set up erddap search for this server:
    e = ERDDAP(
    server=FF['sites'][jj], 
    protocol="tabledap", # Want table data (not a grid map of data) 
    response="csv") #in csv format for pandas

    # now check if buoy data exists for these granules\n",
    e = ERDDAP(server=FF['url'][jj],
               protocol="tabledap",
               response="csv")
    e.dataset_id = FF['sites'][jj]

    for t in icesat_times:
        # add a time buffer (+/-1 hours) to search for relevant buoy data for each granule,
        t_start = (t-timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
        t_end = (t+timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
        e.constraints = {"time>=": t_start,
                        "time<=": t_end}

        # try to download the associated buoy data
        try:
            buoy = e.to_pandas(parse_dates=True)
        except:
            continue
        # if buoy data exists, download the ATL03 photons in the bounding box at this time
        print('downloading ATL03 for '+ e.dataset_id + ' ' + str(t_start))
        # Build ATL03 Request
        poly = buoy_bound_box(FF['geospatial_lat_min'][jj],
                              FF['geospatial_lon_min'][jj],
                              10)
        parms = {"poly": poly,
                 "t0": t_start,
                 "t1": t_end,
                 "srt": icesat2.SRT_OCEAN}        
        atl_gdb = icesat2.atl03sp(parms)
        print('no. of photons: '+str(len(atl_gdb)))


downloading ATL03 for mpoPmzaVikingBuoysTimeseries 2020-09-03T13:44:09+00:00


Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Unable to complete request due to errors


no. of photons: 0
downloading ATL03 for mpoPmzaVikingBuoysTimeseries 2021-03-04T05:03:53+00:00


Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Unable to complete request due to errors


no. of photons: 0
downloading ATL03 for mpoPmzaVikingBuoysTimeseries 2021-05-07T14:15:11+00:00


...

no. of photons: 0
downloading ATL03 for Gulf-of-Mexico-C12-OCSv3-Data-10252018-to-01292020 2019-09-17T03:57:05+00:00


Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Unable to complete request due to errors


no. of photons: 0
downloading ATL03 for Gulf-of-Mexico-C12-OCSv3-Data-10252018-to-01292020 2019-12-16T23:36:54+00:00


Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Using simplified polygon (for CMR request only!), 5 points using tolerance of 0.01
Unable to complete request due to errors


no. of photons: 0
